In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

num_patients = 10
DEF_EPISODES = 10


In [ ]:
def f(x):
    global num_patients
    num_patients = x
    
interact(f,x=widgets.IntSlider(min=10,max=100000,step=100,value=10))

In [1]:
import sys
sys.path.append('../utils')
sys.path.append('../DQN')


import PatientRecordProcessor as pr
import DQN as dqn
import numpy as np
import tensorflow as tf
import pandas as pd
import icu as icu
from sklearn.cluster import MiniBatchKMeans

    

In [2]:
observ_cols = ['elixhauser','re_admission', 'SOFA', 'SIRS', 'Weight_kg', 'GCS', 'HR',
                'SysBP', 'MeanBP', 'DiaBP', 'RR', 'SpO2',
                'Temp_C', 'FiO2_1', 'Potassium', 'Sodium', 'Chloride',
                'Glucose', 'BUN', 'Creatinine', 'Magnesium', 'Calcium',
                'Ionised_Ca', 'CO2_mEqL', 'SGOT', 'SGPT', 'Total_bili',
                'Albumin', 'Hb', 'WBC_count', 'Platelets_count', 'PTT',
                'PT', 'INR', 'Arterial_pH', 'paO2', 'paCO2',
                'Arterial_BE', 'Arterial_lactate', 'HCO3', 'PaO2_FiO2',
                'output_total', 'output_4hourly',
                'sedation', 'mechvent', 'rrt']
n_clusters = 2000
n_patients = 10000

In [3]:
config_path = "config.json"
icu = icu.ICUsim(n_patients)
icu.load_config(config_path)


In [4]:
df = icu.patients
observations = df[observ_cols]


In [5]:
observations.head()

,elixhauser,re_admission,SOFA,SIRS,Weight_kg,GCS,HR,SysBP,MeanBP,DiaBP,...,paCO2,Arterial_BE,Arterial_lactate,HCO3,PaO2_FiO2,output_total,output_4hourly,sedation,mechvent,rrt
0,10,0,1,0,122.047011,11,2,104,76,52,...,88.456388,-7.649027,48.653355,95,1627.918763,20443,6599,0,0,1
1,5,0,10,0,232.205859,7,156,39,68,1,...,51.180712,18.055249,27.720236,66,3098.118663,13607,639,0,1,0
2,7,0,7,3,210.501895,0,36,29,44,18,...,96.395684,35.593605,102.306745,106,2319.571446,225026,6528,1,0,0
3,6,0,11,2,6.819682,13,90,160,44,191,...,37.800088,30.849374,144.170482,77,553.973195,41190,4557,0,1,0
4,0,1,1,0,93.151338,5,172,176,19,2,...,89.127622,5.122725,29.484560,37,2147.021832,95137,5654,1,1,0


In [6]:
mbk = MiniBatchKMeans(init='k-means++', n_clusters=n_clusters,
                        batch_size=100,n_init=10, max_no_improvement=10,
                        verbose=0, init_size=3*n_clusters,random_state=0)
mbk.fit(observations)
clusters = mbk.cluster_centers_


In [9]:
len(clusters)

2000